# Calculate RMSE

In [1]:
import os
import sys
import yaml
from glob import glob
from datetime import datetime, timedelta

import numpy as np
import xarray as xr
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
sys.path.insert(0, os.path.realpath('../libs/'))
import verif_utils as vu

In [4]:
config_name = os.path.realpath('verif_config.yml')

with open(config_name, 'r') as stream:
    conf = yaml.safe_load(stream)

## Verification setup

In [5]:
# ---------------------------------------------------------------------------------------- #
# ERA5 verif target
filename_ERA5 = sorted(glob(conf['ERA5_ours']['save_loc']))

# pick years
year_range = [1990, 2019]
years_pick = np.arange(year_range[0], year_range[1]+1, 1).astype(str)
filename_ERA5 = [fn for fn in filename_ERA5 if any(year in fn for year in years_pick)]

# merge yearly ERA5 as one
ds_ERA5 = [vu.get_forward_data(fn) for fn in filename_ERA5]
ds_ERA5_merge = xr.concat(ds_ERA5, dim='time')

In [6]:
variable_levels = {
    'evaporation': None,
    'total_precipitation': None,
    'surface_net_solar_radiation': None,
    'surface_net_thermal_radiation': None,
    'surface_sensible_heat_flux': None,
    'surface_latent_heat_flux': None,
    'top_net_solar_radiation': None,
    'top_net_thermal_radiation': None,
}

In [7]:
ds_ERA5_merge = vu.ds_subset_everything(ds_ERA5_merge, variable_levels)

In [8]:
def accum_6h_24h(ds_ours, ini=0, copy=True):
    h_shift = ini + 6
    h_convert_ending_time = 24 + ini
    
    if copy:
        ds_ours_shift = ds_ours.copy(deep=True)
        # convert to start time to work with xarray resample
        ds_ours_shift['time'] = ds_ours_shift['time'] - pd.Timedelta(hours=h_shift)
        # accumulate
        ds_ours_24h = ds_ours_shift.resample(time='24h').sum()
    else:
        ds_ours['time'] = ds_ours['time'] - pd.Timedelta(hours=h_shift)
        ds_ours_24h = ds_ours.resample(time='24h').sum()
        
    ds_ours_24h['time'] = ds_ours_24h['time'] + pd.Timedelta(hours=h_convert_ending_time)
    
    return ds_ours_24h

In [9]:
ds_ERA5_merge_24h = accum_6h_24h(ds_ERA5_merge, copy=False)

In [10]:
ds_ERA5_mean = ds_ERA5_merge_24h.mean(dim='time')

In [11]:
path_verif = conf['ERA5_weatherbench']['save_loc_clim']+'ERA5_clim_1990_2019_daily_1deg_interp.nc'
# ds_ERA5_mean.to_netcdf(path_verif, compute=True, format='NETCDF4')

In [13]:
# ds_ERA5_mean.to_netcdf(path_verif, compute=True, format='NETCDF4')